<a href="https://colab.research.google.com/github/Kavin4240/Machine-Learning-Projects/blob/main/Pytorch_vs_Keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision.datasets import MNIST
import tensorflow as tf
from torch.utils.data import DataLoader
from tensorflow.keras import layers,models
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

In [2]:
# Load and preprocess the MNIST dataset using PyTorch
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
train_dataset = MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = MNIST(root='./data', train=False, download=True, transform=transform)

100%|██████████| 9912422/9912422 [00:00<00:00, 107639103.18it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 81627826.03it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 31381238.10it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 3662154.70it/s]


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [3]:
train_loader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=64, shuffle=False)

In [4]:
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

11490434/11490434 [==============================] - 0s 0us/step


In [5]:
train_images = train_images.reshape((60000, 28, 28, 1)).astype('float32') / 255
test_images = test_images.reshape((10000, 28, 28, 1)).astype('float32') / 255

In [6]:
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

In [7]:
# Define the neural network using PyTorch
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(28 * 28, 128)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(128, 10)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.softmax(x)
        return x


In [8]:
model_pt = Net()
criterion = nn.CrossEntropyLoss()
optimizer_pt = optim.Adam(model_pt.parameters(), lr=0.001)

In [9]:
import time
start_time_tf=time.time()
model_tf = models.Sequential()
model_tf.add(layers.Flatten(input_shape=(28, 28, 1)))
model_tf.add(layers.Dense(128, activation='relu'))
model_tf.add(layers.Dense(10, activation='softmax'))

In [10]:
model_tf.compile(optimizer='adam',
                 loss='categorical_crossentropy',
                 metrics=['accuracy'])

In [11]:
model_tf.fit(train_images, train_labels, epochs=5, batch_size=64, validation_split=0.2)
end_time_tf = time.time()

Epoch 1/5
750/750 [==============================] - 7s 8ms/step - loss: 0.3395 - accuracy: 0.9060 - val_loss: 0.1925 - val_accuracy: 0.9457
Epoch 2/5
750/750 [==============================] - 3s 4ms/step - loss: 0.1561 - accuracy: 0.9549 - val_loss: 0.1370 - val_accuracy: 0.9590
Epoch 3/5
750/750 [==============================] - 3s 4ms/step - loss: 0.1083 - accuracy: 0.9689 - val_loss: 0.1118 - val_accuracy: 0.9672
Epoch 4/5
750/750 [==============================] - 3s 5ms/step - loss: 0.0817 - accuracy: 0.9762 - val_loss: 0.0986 - val_accuracy: 0.9693
Epoch 5/5
750/750 [==============================] - 4s 5ms/step - loss: 0.0647 - accuracy: 0.9810 - val_loss: 0.0945 - val_accuracy: 0.9710


In [12]:
print(f"TensorFlow Training Time: {end_time_tf - start_time_tf} seconds")

TensorFlow Training Time: 23.531285285949707 seconds


In [13]:
start_time_pt = time.time()
for epoch in range(5):
    for images, labels in train_loader:
        optimizer_pt.zero_grad()
        outputs = model_pt(images.view(-1, 28 * 28))
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer_pt.step()
end_time_pt = time.time()
print(f"PyTorch Training Time: {end_time_pt - start_time_pt} seconds")

PyTorch Training Time: 87.87143111228943 seconds


In [14]:
# Evaluation using TensorFlow
test_loss_tf, test_acc_tf = model_tf.evaluate(test_images, test_labels)
print(f"TensorFlow Test Accuracy: {test_acc_tf}")

313/313 [==============================] - 1s 2ms/step - loss: 0.0862 - accuracy: 0.9737
TensorFlow Test Accuracy: 0.9736999869346619


In [15]:
# Evaluation using PyTorch
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        outputs = model_pt(images.view(-1, 28 * 28))
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy_pt = correct / total
print(f"PyTorch Test Accuracy: {accuracy_pt}")

PyTorch Test Accuracy: 0.9582
